# HW6 BOOSTING, STACKING, BLENDING

# Импорт библиотек

In [ ]:
from IPython.display import clear_output
!pip install lightgbm==3.0
!pip install gdown
!pip install optuna
!pip install catboost
clear_output()

In [2]:
import catboost
import itertools
import imblearn
import lightgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import scipy
import seaborn as sns
import sklearn
import sklearn.datasets
import xgboost

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, cross_validate, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score, balanced_accuracy_score, average_precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import matthews_corrcoef

# Задание 1. Feature selection и feature engineering

В этом задании используя датасет с [данными о здоровье плода](https://www.kaggle.com/andrewmvd/fetal-health-classification), сравните различные подходы к отбору признаков. Загрузим датасет:

In [ ]:
data = pd.read_csv("data/fetal_health.csv")
data.head()

Посмотрим на данные. Видно, что среди представленных признаков довольно много скоррелированных:

In [ ]:
x = data.drop('fetal_health', axis=1)
x = pd.DataFrame(StandardScaler().fit_transform(x), columns=x.columns, index= x.index)
y = data['fetal_health']

corr_matrix = x.corr(method='spearman')
res = sns.clustermap(corr_matrix, method='weighted', figsize=(10, 10), cmap='coolwarm')

Используя жадный отбор признаков, Add Del и PCA, отберите/сгенерируйте от 1 до 20 признаков и сравните на кросс-валидации accuracy для моделей RandomForestClassifier(n_estimators=50, random_state=42) из sklearn.ensemble, получающихся в процессе отбора/генерации признаков. Сравните полученные результаты с accuracy модели, обученной с использованием всех имеющихся признаков (на кросс-валидации с тем же количеством фолдов). Изобразите результаты в виде графика, опишите свои наблюдения.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

## ENTER YOUR CODE HERE ##

_Вывод:_

_Ваш текст тут._

## Формат результата

* Accuracy модели, обученной с использованием всех имеющихся признаков >0.9.

* Пример графика для одного из пунктов задания:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/Exercises/EX04/result_3_task_ex04_fix.png" width="300">

# Задание 2. Ансамблевое обучение *

В данной задаче вам нужно диагностировать сердечное заболевание у людей по [медицинским показателям](https://www.kaggle.com/datasets/cherngs/heart-disease-cleveland-uci).

In [3]:
heart_dataset = pd.read_csv("data/heart.csv ")

In [ ]:
x = heart_dataset.drop("target", axis=1)
y = heart_dataset['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

Обучите разнообразные классификаторы, приведенные ниже, а также ансамбль `VotingClassifier` из `sklearn.ensemble`, объединяющий эти классификаторы с помощью жесткого или мякого голосования (параметр `voting =` `'hard'` или `'soft'` соответственно). Оцените качество моделей с помощью кросс-валидации на тренировочном наборе, используя функцию `cross_val_score` и метрику `f1`.

In [ ]:
rng = np.random.RandomState(42)

dt = DecisionTreeClassifier(random_state=rng, max_depth=10, min_samples_leaf=10)
rf = RandomForestClassifier(n_estimators=50, random_state=rng)
etc = ExtraTreesClassifier(random_state=rng)
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
svc_lin = SVC(kernel='linear', probability=True, random_state=rng)
svc_rbf = SVC(kernel='rbf', probability=True, random_state=rng)
cat = catboost.CatBoostClassifier(verbose=0, random_seed=42)
lgbm = lightgbm.LGBMClassifier(random_state=42)
lgbm_rf = lightgbm.LGBMClassifier(boosting_type="rf", bagging_freq=1, bagging_fraction=0.7, random_state=42)
xgb = xgboost.XGBClassifier(random_state=42)
xgb_rf = xgboost.XGBRFClassifier(random_state=42)
lr = LogisticRegression(solver='liblinear', max_iter=10000)
nb = GaussianNB()

# Your code here

voting_hard =
voting_soft =
# -----------

for model in [dt, rf, cat, etc, knn, svc_lin, svc_rbf,  xgb, lgbm,  xgb_rf, lgbm_rf, lr, nb, voting_hard, voting_soft]:
    scores = cross_val_score(model, x_train, y_train, cv=KFold(n_splits=3, shuffle=True, random_state=rng), scoring='f1')
    print(f'{model.__class__.__name__}: {scores.mean()}')

Вы можете заметить, что ансамбль показывает хорошее, но не лучшее качество предсказания, попробуем его улучшить. Как вы знаете, ансамбли работают лучше, когда модели, входящие в них не сккоррелированы друг с другом. Определите корреляцию предсказаний базовых моделей в ансамбле на тренировочном наборе, и удалите из ансамбля те модели, чьи предсказания будут сильнее коррелировать с остальными. Можете модифицировать функцию `base_model_pair_correlation` из предыдущего задания.

In [ ]:
# Your code here

Создайте новый ансамбль, на исправленном наборе моделей и оцените его качество с помощью кросс-валидации на тренировочном наборе, используя функцию `cross_val_score` и метрику `f1`.

In [ ]:
# Your code here

voting_hard_2 =
voting_soft_2 =
# ------------

for model in [voting_hard_2, voting_soft_2]:
    scores = cross_val_score(model, x_train, y_train, cv=KFold(n_splits=3, shuffle=True, random_state=rng), scoring='f1')
    print(f'{model.__class__.__name__}: {scores.mean()}')

Обучите все получившиеся модели на тренировочном наборе и испытайте их качество на тестовом наборе. Получилось ли у улучшенных версий ансамблевого классификатора превзойти базовые модели, входящие в него и свои предыдущие версии?

In [ ]:
# Your code here

Какие ансамбли работают лучше? Всегда ли больше моделей значит лучше?

**Напишите вывод**

## Формат результата

Получить значения качества для ансамблей и моделей.